In [2]:
import pytube
from llama3 import Llama3
from langchain import PromptTemplate, LLMChain

In [6]:
# get transcript of youtube vid
def get_transcript(vid_url):
    yt_obj = pytube.YouTube(vid_url)
    trans = yt_obj.captions.get_by_language_code('en') # english
    if trans:
        return trans.generate_srt_captions()
    else:
        raise ValueError("No transcript available")

In [7]:
# llama3 and langchain used to summarize transcript
def summarize_trans(vid_trans):
    model = Llama3(model_name='llama3')

    prompt_tpl = PromptTemplate(
        template="summarize the following text:\n\n{text}\n\nsummary:",
        input_variables=["text"]
    )

    summ_chain = LLMChain(
        llm=model,
        prompt_template=prompt_tpl
    )

    return summ_chain.run(text=vid_trans)

In [15]:
# feed query and get response from model
def response(vid_trans, qry):
    llm_model = Llama3(model_name='llama3')

    # setup a prompt template for answering questions
    prompt_tpl = PromptTemplate(
        template="here is some text:\n\n{text}\n\nbased on the above text, answer this question:\n\nquestion: {question}\n\nanswer:",
        input_variables=["text", "question"]
    )

    # create a chain for question answering
    qa_chain = LLMChain(
        llm=llm_model,
        prompt_template=prompt_tpl
    )

    # generate the answer
    ans = qa_chain.run(text=vid_trans, question=qry)
    return ans

In [17]:
# main function
def main(vid_url, qry_list):
    vid_trans = get_transcript(vid_url)

    summ = summarize_trans(vid_trans)
    print("summary of youtube video: ", summ)

    for qry in qry_list:
        print(f"\nquestion: {qry}\nanswer: {response(vid_trans, qry)}")

In [18]:
video_url = "https://www.youtube.com/watch?v=9gGnTQTYNaE"
questions = [
    "What does this video discuss?",
    "Who is the speaker in the video?",
    "Who is the target audience?",
]

main(video_url, questions)


question: What does this video discuss?
answer: This video explores the concept of machine learning, its distinctions from artificial intelligence (AI) and deep learning, and how businesses are currently utilizing it to enhance operations and drive innovation.

question: Who is the speaker in the video?
answer: The speaker in the video is Luv Aggarwal with IBM Cloud.

question: Who is the target audience?
answer: The target audience is not directly stated but may be professionals, business leaders, and individuals interested in understanding machine learning's applications and implications in modern industries.
